In [ ]:
import os
import time
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from matplotlib import pyplot as plt
from joblib import Parallel, delayed
from sklearn.decomposition import NMF
np.random.seed(1000000)

In [ ]:
d_ = 3
d = d_ ** 2

complete_rating = np.array(pd.read_csv('complete_ratings'))
complete_rating = complete_rating[:100, 1:51] / 5

model = NMF(n_components = d_, init = 'nndsvda', max_iter = 10000)
W = model.fit_transform(complete_rating)
H = model.components_

In [ ]:
def generate_data(W, H, d):
    
    phi_true = np.zeros((W.shape[0], H.shape[1], d))
    psi_true = np.zeros((W.shape[0], H.shape[1], d))
            
    theta_true = np.eye(int(np.sqrt(d))).ravel()
    theta_norm = np.linalg.norm(theta_true)
    theta_true = theta_true / theta_norm
    
    for i in range(W.shape[0]):
        
        for j in range(H.shape[1]):
            
            temp = np.outer(W[i], H[:, j])
            phi_true[i, j] = temp.ravel()
            psi_true[i, j] = phi_true[i, j] + 1e-1 * np.random.normal(0, 1, d)
            
    max_value = np.max(np.dot(phi_true, theta_true))
    phi_true = phi_true / max_value
    psi_true = psi_true / max_value
            
    return phi_true, psi_true, theta_true

In [ ]:
phi_true, psi_true, theta_true = generate_data(W, H, d)
phi_true.shape, psi_true.shape, theta_true

In [ ]:
# calculate the beta
def get_beta(rho, delta, V_bar, theta_true):
    
    ld = 1
    beta = rho * np.sqrt(2 * np.log((np.sqrt(np.linalg.det(V_bar)) * (np.linalg.det(ld * np.eye(d)) ** (-1/2))) / delta)) + np.sqrt(ld) * np.linalg.norm(theta_true)
    
    return beta

In [ ]:
# slsqp
def get_decision(psi_action, theta_hat, V_bar, beta, d):
    
    def maximize_reward(theta, psi_a):

        return -1.0 * psi_a.dot(theta)
    
    # constraint confidence set: || theta_hat - theta ||_V_bar < Beta
    def constraint(theta, theta_hat, V_bar, beta):
        
        temp = np.array(theta_hat - theta.reshape(-1, 1))
        norm = np.sqrt(temp.reshape(-1, 1).T.dot(V_bar).dot(temp.reshape(-1, 1)))
        
        return beta - norm[0][0]
    
    # constraints for optimization
    beta_constraint = {'type': 'ineq', 'fun' : lambda theta: constraint(theta, theta_hat, V_bar, beta)}
    
    theta_list = []
    reward_list = []
    
    for psi_a in psi_action:
        
        res = minimize(maximize_reward, x0 = np.ones(d), args = (psi_a), method = 'SLSQP', constraints = [beta_constraint], options = {'ftol': 1e-3, 'eps': 1e-10, 'maxiter': 1e6, 'disp': False})
        theta_list.append(res.x)
        reward_list.append(psi_a.dot(res.x))
        # print(res.message)
        
    decision = np.argmax(reward_list)
    theta_tilde = theta_list[decision]
    
    return decision, theta_tilde

In [ ]:
def prepare_plot_data(data_list, trials):
    
    result_lists = []
    
    for i in range(0, len(data_list), trials):
        
        subset = data_list[i : i + trials]
        sum_array = np.sum([np.array(item[-1]) for item in subset], axis=0)
        average_array = sum_array / trials
        result_lists.append(average_array.tolist())
    
    return result_lists

In [ ]:
R = 0.1
ld = 1
trials = 1
alpha = 0.5
num_agent = [3, 5]
baseline_idx = 40
delta_value = 1e-3
iterations = 800000
every_num_point = 25
optimal_alg = []
reward_alg = []
baseline_alg = []
cummulative_regret_alg = []
cumulative_violate_alg = []
cumulative_baseline_alg = []
L = np.max(np.linalg.norm(phi_true, axis = 2))

In [ ]:
phi_true_1 = phi_true.copy()
phi_true_1[:, :, -1] = 0

phi_true_2 = phi_true.copy()
phi_true_2[:, :, 4] = 0

phi_true_3 = phi_true.copy()
phi_true_3[:, :, 0] = 0

phi_true_group = np.array([phi_true, phi_true_1, phi_true_2, phi_true_3] * 3)

In [ ]:
psi_true_1 = psi_true.copy()
psi_true_1[:, :, -1] = 0

psi_true_2 = psi_true.copy()
psi_true_2[:, :, 4] = 0

psi_true_3 = psi_true.copy()
psi_true_3[:, :, 0] = 0

psi_true_group = np.array([psi_true, psi_true_1, psi_true_2, psi_true_3] * 3)

In [ ]:
rewards_group = np.array([phi_true_group[i].dot(theta_true) for i in range(np.shape(phi_true_group)[0])])

In [ ]:
# generate the index set for context
sample_id = np.random.randint(0, np.shape(phi_true)[0], size=(trials, iterations))

# generate the noise of reward for each iteration
noise = np.random.normal(0, 1e-3, size = (num_agent[-1], trials, iterations))

# calculate r_l and r_h
product_results = np.dot(phi_true_group, theta_true)
sorted_values = np.sort(product_results, axis=2)
all_baseline_values = sorted_values[:, :, -baseline_idx]
r_h = np.max(all_baseline_values)
r_l = np.min(all_baseline_values)
            
# gererate rho_bar
rho_bar = np.random.uniform(1e-10, alpha * r_l / (np.linalg.norm(theta_true) + r_h), size = (trials, iterations))

# generate zeta
zeta_data = np.random.normal(0, 1e-3, (trials, iterations, len(theta_true)))
zeta_zero = zeta_data - np.mean(zeta_data, axis = 2, keepdims = True)
zeta = zeta_zero / np.linalg.norm(zeta_zero, axis = 2, keepdims = True)

In [ ]:
# our setting
start = time.time()

for M in num_agent:
    
    for T in range(trials):
        
        total_regret = 0
        total_reward = 0
        total_violate = 0
        total_baseline = 0
        cummulative_regret = []
        cummulative_violate = []
        cummulative_baseline = []
        optimal_list = []
        reward_list = []
        baseline_list = []
        
        t_last = 0
        V_last = ld * np.eye(d)
        W_syn = np.zeros((d, d))
        U_syn = np.zeros((d, 1))
        
        W_new_list = [np.zeros((d, d)) for i in range(M)]
        U_new_list = [np.zeros((d, 1)) for i in range(M)]
        
        B = (iterations * np.log(M * iterations)) / (d * M)
#         print('B is equal to', B)
        
        syn = 0

        for t in range(1, iterations + 1): 
            
#             print('')
            print('Trial: {} t: {}:'.format(T,t))
            
            index = sample_id[T][t-1]
            
            for i in range(M):
                
                phi = phi_true_group[i][index]
                psi = psi_true_group[i][index]
                original = rewards_group[i][index]

                x_star = np.argmax(np.dot(np.array(psi), theta_true))
                optimal = original[x_star]
#                 print('best decision is:', np.argmax(original))
                
                x_b = np.argsort(original)[::-1][baseline_idx]
                r_b = original[x_b]
#                 print('baseline decision is:', x_b)
                
                V_bar = ld * np.eye(d) + W_syn + W_new_list[i]
                theta_hat = np.dot(np.linalg.inv(V_bar), (U_syn + U_new_list[i]))

                # construct the confidence ellipsoid beta
                beta = get_beta(rho = np.sqrt(1 + R ** 2), delta = delta_value / 2, V_bar = V_bar, theta_true = theta_true)
                
                #construct the trimmed action set
                tas = (psi.dot(theta_hat) >= beta * L / np.sqrt(np.min(np.linalg.eigvals(V_bar))) + (1 - alpha) * r_b)
                phi_set = phi[tas.ravel()]
                psi_set = psi[tas.ravel()]
                original_set = original[tas.ravel()]
                
                # get the best action
                if (psi_set.size != 0) and (np.min(np.linalg.eigvals(V_bar)) >= np.square(2 * L * beta / ((optimal - r_h) + alpha * r_b))):
                
                    decision, theta_tilde = get_decision(psi_set, theta_hat, V_bar, beta, d)
                    psi_new = psi_set[decision]
                    y = original_set[decision]
#                     print("play learner's decision:", decision)
                    
                else:
                    
                    decision = x_b
                    total_baseline += 1
                    psi_new = (1 - rho_bar[T][t-1]) * psi[decision] + rho_bar[T][t-1] * zeta[T][t-1]
                    y = (1 - rho_bar[T][t-1]) * original[decision] + rho_bar[T][t-1] * np.dot(zeta[T][t-1], theta_true)
#                     print("play conservative decision:", decision)
                    
                regret = optimal - y
                total_regret = total_regret + regret
                total_reward = total_reward + y

                # update W_new and U_new
                W_new_list[i] = W_new_list[i] + np.outer(psi_new, psi_new)
                U_new_list[i] = U_new_list[i] + psi_new.reshape(-1, 1) * (y + noise[i][T][t-1])
                V = ld * np.eye(d) + W_syn + W_new_list[i]
                
                LHS_condition = np.log(np.linalg.det(V) / np.linalg.det(V_last)) * (t - t_last)
                
                if LHS_condition >= B:
                    
#                     print('synchronization start for agent', i)
#                     print('LHS condition is:', LHS_condition)
                    
                    syn = 1
                    
#                 print('----------')
                
                if y < (1 - alpha) * r_b:
                    
                    total_violate += 1
#                     print('violate the constraint')
                
            if syn == 1:
                
                W_syn = W_syn + np.sum(W_new_list, axis=0)
                U_syn = U_syn + np.sum(U_new_list, axis=0)
                
                W_new_list = [np.zeros((d, d)) for i in range(M)]
                U_new_list = [np.zeros((d, 1)) for i in range(M)]
                t_last = t
                V_last = ld * np.eye(d) + W_syn
                
                syn = 0
                
#             print('cummulative_regret is: ', total_regret)
            cummulative_regret.append(total_regret)
            cummulative_violate.append(total_violate)
            cummulative_baseline.append(total_baseline)
            optimal_list.append(np.max(np.dot(phi, theta_true)))
            reward_list.append(y)
            baseline_list.append((1 - alpha) * r_b)
            
        cummulative_regret_alg.append((M, T, cummulative_regret))
        cumulative_violate_alg.append((M, T, cummulative_violate))
        cumulative_baseline_alg.append((M, T, cummulative_baseline))
        optimal_alg.append((M, T, optimal_list))
        reward_alg.append((M, T, reward_list))
        baseline_alg.append((M, T, baseline_list))
        
end = time.time()
print('Finished! The total time we use is: ', end - start)

In [ ]:
plot_lists = prepare_plot_data(cummulative_regret_alg, trials)
x_value = np.array([i for i in range(len(plot_lists[0]))])
y_1 = [plot_lists[0][i] / num_agent[0] for i in range(len(plot_lists[0]))]
y_2 = [plot_lists[1][i] / num_agent[1] for i in range(len(plot_lists[1]))]

plt.figure(figsize=(12,8), dpi=300)

colors = (['black', 'blue', 'darkgreen', 'purple', 'darkred', 'grey'])
markers = ['*', 's', 'o', 'X', '^', 'P']

plt.rcParams['xtick.labelsize'] = 25
plt.rcParams['ytick.labelsize'] = 25
plt.rc('legend', fontsize = 25)

ax = plt.gca()
ax.ticklabel_format(style='sci', axis='both', scilimits=(0, 0), useOffset=False)

plt.plot(x_value, y_1, label = 'M = 3', color = colors[1], linewidth=3)
plt.scatter(x_value[::50000], y_1[::50000], label = 'M = 3', marker = markers[1], color = colors[1], s=300)
plt.plot(x_value, y_2, label = 'M = 5', color = colors[2], linewidth=3)
plt.scatter(x_value[::50000], y_2[::50000], label = 'M = 5', marker = markers[2], color = colors[2], s=300)

legend_elements = [mlines.Line2D([0], [0], color=colors[1], lw = 5, label = 'M = 3', marker = markers[1], markersize = 15), 
                   mlines.Line2D([0], [0], color=colors[2], lw = 5, label = 'M = 5', marker = markers[2], markersize = 15)]

plt.grid(True)
plt.xlabel('round,t', fontsize=25)
plt.ylabel('per-agent cumulative regret Rt', fontsize=25)
plt.title('movielens data', fontsize=25)
plt.legend(handles=legend_elements)
# plt.savefig('plot_8.pdf', dpi=600, bbox_inches = 'tight')
plt.show()
plt.close()